In [1]:
import pickle

import pandas as pd

In [2]:
df = pd.read_csv('./user_item_title_237742i.csv', index_col=0, parse_dates=['event_datetime'])
df.shape

(237742, 4)

In [3]:
df.head()

,user_id,item_id,title,event_datetime
0,user_4329458,video_3442132,Сопрано S03E06 Университет - Разбор Гоблина с ...,2023-08-31 21:00:03
1,user_3576973,video_3131219,#видеомейкинг #видеоурок,2023-08-31 21:00:08
2,user_4161636,video_2173882,Первые высотки проекта (Интерьер),2023-08-31 21:00:18
3,user_25263908,video_2361603,Волк кушает бублики,2023-08-31 21:01:02
4,user_17492633,video_1682383,Рисунок поросенка Мейбл - Легкие рисунки - Мил...,2023-08-31 21:01:46


In [4]:
df['event_datetime'].dt.date.value_counts().sort_index().tail(14)

event_datetime
2023-12-18    2031
2023-12-19    2280
2023-12-20    1949
2023-12-21    2006
2023-12-22    2255
2023-12-23    1807
2023-12-24    1866
2023-12-25    2287
2023-12-26    2002
2023-12-27    1884
2023-12-28    1944
2023-12-29    2143
2023-12-30    1873
2023-12-31    1412
Name: count, dtype: int64

In [5]:
df_train = df[df['event_datetime'] < '2023-12-25']
df_test = df[df['event_datetime'] >= '2023-12-25']
df_test = df_test[df_test['user_id'].isin(df_train['user_id'])]
df_train.shape, df_test.shape

((224197, 4), (12626, 4))

In [6]:
len(set(df_train['user_id']) & set(df_test['user_id'])), df_test['user_id'].nunique()

(2067, 2067)

In [7]:
df_test.groupby('user_id')['item_id'].nunique().describe()

count    2067.000000
mean        6.108370
std        31.827461
min         1.000000
25%         1.000000
50%         2.000000
75%         4.000000
max       667.000000
Name: item_id, dtype: float64

In [8]:
train_sequences = df_train.sort_values('event_datetime').groupby('user_id')['item_id'].apply(list).reset_index().explode('item_id')
train_items_set = set(train_sequences['item_id'])
# Возьму только первый известный айтем тестовой недели, так как прогнозируе следующий айтем
def return_train_seen_items(x):
    for item_id in x.values:
        if item_id in train_items_set:
            return item_id
    else:
        return None


test_sequences = df_test.sort_values('event_datetime').groupby('user_id')['item_id'].apply(return_train_seen_items).reset_index()
test_sequences.head()

,user_id,item_id
0,user_10024788,video_82960
1,user_10049908,video_1849076
2,user_10063731,None
3,user_10070433,video_1860820
4,user_10078413,video_3382395


In [9]:
test_sequences = test_sequences[~test_sequences['item_id'].isna()]
test_sequences.head()

,user_id,item_id
0,user_10024788,video_82960
1,user_10049908,video_1849076
3,user_10070433,video_1860820
4,user_10078413,video_3382395
7,user_10103094,video_2775359


In [10]:
with open('../arfifacts/user_oe.pkl', 'rb') as file:
    user_oe = pickle.load(file)
    
with open('../arfifacts/item_oe.pkl', 'rb') as file:
    item_oe = pickle.load(file)

train_sequences['user_id'] = user_oe.transform(train_sequences['user_id'].values.reshape(-1, 1))
train_sequences['item_id'] = item_oe.transform(train_sequences['item_id'].values.reshape(-1, 1))

test_sequences['user_id'] = user_oe.transform(test_sequences['user_id'].values.reshape(-1, 1))
test_sequences['item_id'] = item_oe.transform(test_sequences['item_id'].values.reshape(-1, 1))

In [11]:
train_sequences

,user_id,item_id
0,0,17863
0,0,14687
0,0,88
0,0,14128
0,0,14073
...,...,...
9216,9379,18216
9216,9379,20483
9216,9379,16770
9216,9379,5847


In [12]:
test_sequences

,user_id,item_id
0,8,22098
1,16,6064
3,25,6142
4,27,16762
7,36,12476
...,...,...
2061,9350,15730
2063,9365,10367
2064,9369,14402
2065,9375,1587


In [13]:
# Save to txt
train_sequences.to_csv(
    '../aprec/data/bert4rec/rutube_week_train.txt',
    sep=' ',
    header=False,
    index=False
)
test_sequences.to_csv(
    '../aprec/data/bert4rec/rutube_week_test.txt',
    sep=' ',
    header=False,
    index=False
)